In [5]:
import pandas as pd
import pathlib2 as pathlib

In [32]:
from optuna import Trial, create_study
from optuna.pruners import SuccessiveHalvingPruner
# import sklearn
from sklearn.pipeline import Pipeline
# from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.decomposition import LatentDirichletAllocation
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
# from sklearn.metrics import make_scorer
from sklearn.model_selection import train_test_split

from sklearn.model_selection import cross_val_score, KFold
from sklearn.model_selection import GridSearchCV

from gensim.models.coherencemodel import CoherenceModel
import gensim.corpora as corpora
from gensim.parsing.preprocessing import STOPWORDS
from gensim.utils import tokenize

import numpy as np
import pandas as pd
import re

import joblib
randomstate=313


In [23]:
cwd=pathlib.Path.cwd()
datapath=cwd.joinpath('data')
datawreviewsfile=datapath.joinpath('processed/innerJoinData.csv')
dataworeviewsfile=datapath.joinpath('raw/collaborative_book_metadata_with_genredummies.csv')
models_path=cwd.joinpath('models')

In [44]:
datadf=pd.read_csv(datawreviewsfile,sep=';')
fulldatadf=pd.read_csv(dataworeviewsfile,sep=';')
fulldatadf=fulldatadf.drop(['biography', 'children', 'comics', 'crime', 'fantasy', 'fiction',
       'graphic', 'historicalfiction', 'history', 'mystery', 'nonfiction',
       'paranormal', 'poetry', 'romance', 'thriller', 'youngadult'],axis=1)

In [5]:
# datadf.head()

In [45]:
fulldatadf.columns

Index(['book_id', 'title', 'image_url', 'url', 'num_pages', 'ratings_count',
       'description', 'genre', 'name', 'book_id_mapping', 'num_genres'],
      dtype='object')

In [9]:
def remove_punctuation(x):
    cleanedtext=re.sub('[^A-Za-z0-9]+', '', x)
    return cleanedtext
fulldatadf['description']=fulldatadf.description.transform(remove_punctuation)

In [25]:
fulldatadf['genre']=fulldatadf.genre.apply(lambda x: x.replace("'","")) # remove '
fulldatadf['genre']=fulldatadf.genre.apply(lambda x: x.replace("-","")) # revome - 
fulldatadf['genre']=fulldatadf.genre.apply(lambda x: x.replace(" ","")) # remove spaces
fulldatadf['genre']=fulldatadf.genre.apply(lambda x: x[1:-1].split(',')) # split into list


In [26]:
fulldatadf.genre

0     [fantasy, paranormal, romance, fiction, histor...
1     [youngadult, fantasy, paranormal, romance, fic...
2                                    [romance, fiction]
3                   [mystery, thriller, crime, fiction]
4                        [fantasy, paranormal, fiction]
                            ...                        
91    [nonfiction, history, historicalfiction, biogr...
92    [comics, graphic, nonfiction, history, histori...
93    [youngadult, fantasy, paranormal, fiction, chi...
94    [fantasy, paranormal, fiction, youngadult, rom...
95    [fantasy, paranormal, fiction, youngadult, chi...
Name: genre, Length: 96, dtype: object

In [28]:
xtrain,xtest=train_test_split(fulldatadf['genre'],test_size=0.1,random_state=randomstate,shuffle=True)

In [29]:
xtrain

32    [fantasy, paranormal, youngadult, romance, fic...
95    [fantasy, paranormal, fiction, youngadult, chi...
75    [comics, graphic, fiction, fantasy, paranormal...
92    [comics, graphic, nonfiction, history, histori...
79    [nonfiction, history, historicalfiction, biogr...
                            ...                        
39                                         [nonfiction]
31    [nonfiction, history, historicalfiction, biogr...
87    [nonfiction, history, historicalfiction, biogr...
42    [fantasy, paranormal, fiction, comics, graphic...
8            [fantasy, paranormal, fiction, youngadult]
Name: genre, Length: 86, dtype: object

In [39]:
mlb=MultiLabelBinarizer()
mlbmodel=mlb.fit(xtrain)
newcols=mlbmodel.classes_
mlbmodelpath=models_path.joinpath('mlb_model')
joblib.dump(mlbmodel,mlbmodelpath.joinpath('mlbmodel.pkl').as_posix())


['c:/Users/lekle/Projects_Code/HA_MaschinellesLernen_MADS23oB/models/mlb_model/mlbmodel.pkl']

In [41]:
newcols

array(['biography', 'children', 'comics', 'crime', 'fantasy', 'fiction',
       'graphic', 'historicalfiction', 'history', 'mystery', 'nonfiction',
       'paranormal', 'poetry', 'romance', 'thriller', 'youngadult'],
      dtype=object)

In [46]:
pred=mlbmodel.transform(fulldatadf.genre)
preddf=pd.DataFrame(pred,columns=newcols)
a=fulldatadf.join(preddf)

c:\Users\lekle\anaconda3\Lib\site-packages\sklearn\preprocessing\_label.py:900: UserWarning: unknown class(es) [' ', "'", ',', '[', ']', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'l', 'm', 'n', 'o', 'p', 'r', 's', 't', 'u', 'y'] will be ignored
  warnings.warn(


In [48]:
a.columns

Index(['book_id', 'title', 'image_url', 'url', 'num_pages', 'ratings_count',
       'description', 'genre', 'name', 'book_id_mapping', 'num_genres',
       'biography', 'children', 'comics', 'crime', 'fantasy', 'fiction',
       'graphic', 'historicalfiction', 'history', 'mystery', 'nonfiction',
       'paranormal', 'poetry', 'romance', 'thriller', 'youngadult'],
      dtype='object')

In [7]:
def tokenize_column(column):
    l=[]
    for x in range(len(column)):
        l.append(list(tokenize(column.iloc[x])))
    return l
fulldatadf['description_list']=tokenize_column(fulldatadf.description)

In [8]:
xtrain,xtest=train_test_split(fulldatadf['description'],test_size=0.1,random_state=randomstate,shuffle=True)

In [9]:
len(xtrain)

86

In [10]:
len(xtest)

10

## group books via genres
- vectorize genre-list, countvectorizer vs tfidf (pos: fiction genre very prevalent, thus less relevant in distinction of books)
    - cluster genres (uneven size clusters? not very distinct? fuzzy clustering instead?)
    - topic modeling (lda)

## group books via blurb content
- vectorize, topics via topic modeling
-  tfidf -> lda
    - evaluate via topic coherence
- https://github.com/kapadias/medium-articles/blob/master/natural-language-processing/topic-modeling/Evaluate%20Topic%20Models.ipynb
- https://medium.com/@walter_sperat/using-optuna-with-sklearn-the-right-way-part-1-6b4ad0ab2451
- https://learn-scikit.oneoffcoder.com/optuna.html
- https://learn-scikit.oneoffcoder.com/gensim.html
- https://stackoverflow.com/questions/60613532/how-do-i-calculate-the-coherence-score-of-an-sklearn-lda-model
-


### description
pipeline: tfidf, lda

- no crossvalidation in optuna when evaluating topics based on gensim coherence c_v, as scikitlearns make_scorer() only works with y_true to evaluate agains (only supervised clustering), this is not ideal, different 

In [11]:
#pipe = Pipeline([('tfidf',TfidfVectorizer()),('lda',LatentDirichletAllocation())])

studyname='description_tfidflda_studydefault'
thismodelpath=models_path.joinpath(f'{studyname}')
try: 
    thismodelpath.mkdir(exist_ok=False)
except FileExistsError:
    print('Directory already exists')


Directory already exists


In [12]:
def topiccoherencescorer(pipe,X):
    n_top_words = 15 #higher value for higher coherence, i.e. more word to make connections for coherence
    topics=pipe.named_steps['lda'].components_
    texts=[[word for word in doc.split()] for doc in X]
    # texts=X
    dictionary=corpora.Dictionary(texts)
    corpus=[dictionary.doc2bow(text) for text in texts]
    feature_names = [dictionary[i] for i in range(len(dictionary))]
    top_words = []
    for topic in topics:
        top_words.append([feature_names[i] for i in topic.argsort()[:-n_top_words - 1:-1]])
    cm = CoherenceModel(topics=top_words, texts=texts, dictionary=dictionary, coherence='c_v')
    return cm.get_coherence()


In [13]:
def inst_tfidf(trial:Trial)->TfidfVectorizer:
    params={
        'norm':trial.suggest_categorical('norm',['l1','l2', None]),
        'smooth_idf':trial.suggest_categorical('smooth_idf',[True,False]),
        'sublinear_tf':trial.suggest_categorical('sublinear_tf',[True,False]),
        'stop_words':trial.suggest_categorical('stop_words',['english',list(STOPWORDS)]),
        'tokenizer':tokenize,
        # 'max_df':trial.suggest_float('max_df',0,1),
        # 'min_df':trial.suggest_float('min_df',0,1),
        'max_features':trial.suggest_categorical('max_features',[None,100,50,25,10]) #can't use int, because of None
        }
    return TfidfVectorizer(**params)
def inst_lda(trial:Trial)->LatentDirichletAllocation:
    params={
        'n_components':trial.suggest_int('n_components',3,20),
        'learning_method':trial.suggest_categorical('learning_method',['batch','online']),
        'learning_decay':trial.suggest_float('learning_decay',0.5,0.9),
        'learning_offset':trial.suggest_float('learning_offset',2,20),
        'max_iter':trial.suggest_int('max_iter',2,20),
        'batch_size':trial.suggest_int('batch_size',5,20),
        'max_doc_update_iter':trial.suggest_int('max_doc_update_iter',0.001,0.1),
        'n_jobs':-1,
        'random_state':randomstate
    }
    return LatentDirichletAllocation(**params)
def inst_pipeTFLDA(trial:Trial)->Pipeline:
    pipeline=Pipeline([
        ('tfidf',inst_tfidf(trial)),
        ('lda',inst_lda(trial))
    ])
    return pipeline

def objective(trial:Trial,x:pd.DataFrame)->float:
    model=inst_pipeTFLDA(trial)
    
    # pipe=model.fit(x)
    # score=topiccoherencescorer(pipe,x) #no cv, as scorer not compatible
    kf = KFold(n_splits=5, shuffle=True, random_state=randomstate)
    scores = cross_val_score(model, x, cv=kf) #using default scorer --> approx log-likelohood

    return np.min([np.mean(scores), np.median(scores)])

In [14]:
storage=thismodelpath.joinpath("description_tfidflda_studydefault.db")
if storage.exists():
    storage.unlink()
else:
    print("No sqlite db found")

In [15]:
study=create_study(study_name=studyname,direction='maximize',storage=f'sqlite:///{storage.as_posix()}',load_if_exists=False) #TPESampler used as default, no pruning

[I 2024-08-28 18:11:46,293] A new study created in RDB with name: description_tfidflda_studydefault


In [16]:
study.optimize(lambda trial: objective(trial,xtrain),n_trials=100,n_jobs=-1,show_progress_bar=True)

  0%|          | 0/100 [00:00<?, ?it/s]

c:\Users\lekle\anaconda3\Lib\site-packages\optuna\distributions.py:524: UserWarning: Choices for a categorical distribution should be a tuple of None, bool, int, float and str for persistent storage but contains ['within', 'by', 'also', 'sometimes', 'before', 'thereafter', 'became', 'already', 'thus', 'whereby', 'but', 'herself', 'these', 'formerly', 'itself', 'though', 'her', 'fill', 'him', 'there', 'noone', 'own', 'meanwhile', 'twenty', 'hereupon', 'anyone', 'in', 'every', 'only', 'forty', 'seemed', 'empty', 'something', 'must', 'why', 'thereupon', 'much', 'kg', 'perhaps', 'everywhere', 'wherein', 'several', 'co', 'de', 'more', 'us', 'were', 'an', 'now', 'namely', 'various', 'unless', 'indeed', 'eleven', 'bill', 'himself', 'per', 'using', 'moreover', 'third', 'sometime', 'into', 'would', 'even', 'cannot', 'can', 'serious', 'seeming', 'during', 'twelve', 're', 'don', 'three', 'really', 'at', 'further', 'nothing', 'without', 'yet', 'i', 'so', 'done', 'former', 'if', 'move', 'after', 'b

[I 2024-08-28 18:11:58,721] Trial 10 finished with value: -1519.7189071200278 and parameters: {'norm': 'l2', 'smooth_idf': True, 'sublinear_tf': True, 'stop_words': 'english', 'max_features': 50, 'n_components': 18, 'learning_method': 'online', 'learning_decay': 0.5548079177528745, 'learning_offset': 12.449405839803532, 'max_iter': 7, 'batch_size': 89, 'max_doc_update_iter': 0}. Best is trial 10 with value: -1519.7189071200278.
[I 2024-08-28 18:11:58,996] Trial 2 finished with value: -617.0064802061343 and parameters: {'norm': 'l2', 'smooth_idf': False, 'sublinear_tf': True, 'stop_words': ['within', 'by', 'also', 'sometimes', 'before', 'thereafter', 'became', 'already', 'thus', 'whereby', 'but', 'herself', 'these', 'formerly', 'itself', 'though', 'her', 'fill', 'him', 'there', 'noone', 'own', 'meanwhile', 'twenty', 'hereupon', 'anyone', 'in', 'every', 'only', 'forty', 'seemed', 'empty', 'something', 'must', 'why', 'thereupon', 'much', 'kg', 'perhaps', 'everywhere', 'wherein', 'several'

In [17]:
joblib.dump(study,thismodelpath.joinpath(f'study_{study.study_name}.pkl').as_posix())

['c:/Users/lekle/Projects_Code/HA_MaschinellesLernen_MADS23oB/models/description_tfidflda_studydefault/study_description_tfidflda_studydefault.pkl']

In [18]:
study.best_params # no stopwords, remove none from trial values

{'norm': 'l1',
 'smooth_idf': True,
 'sublinear_tf': False,
 'stop_words': ['within',
  'by',
  'also',
  'sometimes',
  'before',
  'thereafter',
  'became',
  'already',
  'thus',
  'whereby',
  'but',
  'herself',
  'these',
  'formerly',
  'itself',
  'though',
  'her',
  'fill',
  'him',
  'there',
  'noone',
  'own',
  'meanwhile',
  'twenty',
  'hereupon',
  'anyone',
  'in',
  'every',
  'only',
  'forty',
  'seemed',
  'empty',
  'something',
  'must',
  'why',
  'thereupon',
  'much',
  'kg',
  'perhaps',
  'everywhere',
  'wherein',
  'several',
  'co',
  'de',
  'more',
  'us',
  'were',
  'an',
  'now',
  'namely',
  'various',
  'unless',
  'indeed',
  'eleven',
  'bill',
  'himself',
  'per',
  'using',
  'moreover',
  'third',
  'sometime',
  'into',
  'would',
  'even',
  'cannot',
  'can',
  'serious',
  'seeming',
  'during',
  'twelve',
  're',
  'don',
  'three',
  'really',
  'at',
  'further',
  'nothing',
  'without',
  'yet',
  'i',
  'so',
  'done',
  'former'

In [19]:
model = inst_pipeTFLDA(study.best_trial)
fitpipe=model.fit(fulldatadf.description)
joblib.dump(fitpipe,thismodelpath.joinpath(f'fittedpipelinedefault.pkl').as_posix())

['c:/Users/lekle/Projects_Code/HA_MaschinellesLernen_MADS23oB/models/description_tfidflda_studydefault/fittedpipelinedefault.pkl']

In [20]:
type(fitpipe)

sklearn.pipeline.Pipeline

In [21]:
#save fitted model
trainpred=fitpipe.transform(xtrain)
testpred=fitpipe.transform(xtest)
predcolumn=fitpipe.transform(fulldatadf.description)
predcolumn=[str(col) for col in predcolumn]

In [22]:
fulldatadf['descr_lda']=predcolumn
fulldatadf.to_csv(thismodelpath.joinpath('transformeddatadefault.csv'))

In [23]:
testpred #same distribution for all, model not working

array([[0.33333333, 0.33333333, 0.33333333],
       [0.33333333, 0.33333333, 0.33333333],
       [0.33333333, 0.33333333, 0.33333333],
       [0.33333333, 0.33333333, 0.33333333],
       [0.33333333, 0.33333333, 0.33333333],
       [0.33333333, 0.33333333, 0.33333333],
       [0.33333333, 0.33333333, 0.33333333],
       [0.33333333, 0.33333333, 0.33333333],
       [0.33333333, 0.33333333, 0.33333333],
       [0.33333333, 0.33333333, 0.33333333]])

In [24]:
n_top_words = 15 
topics=fitpipe.named_steps['lda'].components_
texts=[[word for word in doc.split()] for doc in xtrain]
# texts=X
dictionary=corpora.Dictionary(texts)
corpus=[dictionary.doc2bow(text) for text in texts]
feature_names = [dictionary[i] for i in range(len(dictionary))]
top_words = []
for topic in topics:
    top_words.append([feature_names[i] for i in topic.argsort()[:-n_top_words - 1:-1]])

In [25]:
top_words # topics look the same, switch to count vectorizer instead of tfidf?, more preprocessing

[['While',
  'with',
  'J.K.',
  'Later.',
  'his',
  'Francisco',
  'rescue',
  '2016.',
  'night.',
  'street.',
  'After',
  'chance',
  'seventeen-year-old',
  'mercy',
  'angel.'],
 ['with',
  'rescue',
  'Magic,',
  'put',
  'including',
  'night.',
  'situation',
  'survival.',
  'a',
  'who',
  'street.',
  'West',
  'comes',
  'her',
  'It'],
 ['with',
  'children.',
  'John',
  'night.',
  'street.',
  'belongs,',
  'easier',
  '30,',
  'J.K.',
  'Raffe',
  'anything',
  'Jack',
  'chance',
  'twisted',
  'everything']]

### with count vectorizer

In [26]:
studyname='description_countlda_studydefault'
thismodelpath=models_path.joinpath(f'{studyname}')
try: 
    thismodelpath.mkdir(exist_ok=False)
except FileExistsError:
    print('Directory already exists')

In [27]:
def inst_countvec(trial:Trial)->CountVectorizer:
    params={
        'stop_words':trial.suggest_categorical('stop_words',['english',list(STOPWORDS)]),
        'tokenizer':tokenize,
        # 'max_df':trial.suggest_float('max_df',0,1),
        # 'min_df':trial.suggest_float('min_df',0,1),
        'max_features':trial.suggest_categorical('max_features',[None,100,50,25,10]) #can't use int, because of None
        }
    return CountVectorizer(**params)
def inst_lda(trial:Trial)->LatentDirichletAllocation:
    params={
        'n_components':trial.suggest_int('n_components',3,20),
        'learning_method':trial.suggest_categorical('learning_method',['batch','online']),
        'learning_decay':trial.suggest_float('learning_decay',0.5,0.9),
        'learning_offset':trial.suggest_float('learning_offset',2,20),
        'max_iter':trial.suggest_int('max_iter',2,20),
        'batch_size':trial.suggest_int('batch_size',5,20),
        'max_doc_update_iter':trial.suggest_int('max_doc_update_iter',0.001,0.1),
        'n_jobs':-1,
        'random_state':randomstate
    }
    return LatentDirichletAllocation(**params)
def inst_pipeTFLDA(trial:Trial)->Pipeline:
    pipeline=Pipeline([
        ('countvectorizer',inst_countvec(trial)),
        ('lda',inst_lda(trial))
    ])
    return pipeline

def objective(trial:Trial,x:pd.DataFrame)->float:
    model=inst_pipeTFLDA(trial)
    
    # pipe=model.fit(x)
    # score=topiccoherencescorer(pipe,x) #no cv, as scorer not compatible
    kf = KFold(n_splits=5, shuffle=True, random_state=randomstate)
    scores = cross_val_score(model, x, cv=kf) #using default scorer --> approx log-likelohood

    return np.min([np.mean(scores), np.median(scores)])

In [28]:
storage=thismodelpath.joinpath("description_countveclda_studydefault.db")
if storage.exists():
    storage.unlink()
else:
    print("No sqlite db found")

study2=create_study(study_name=studyname,direction='maximize',storage=f'sqlite:///{storage.as_posix()}',load_if_exists=False) #TPESampler used as default, no pruning
study2.optimize(lambda trial: objective(trial,xtrain),n_trials=100,n_jobs=-1,show_progress_bar=True)
joblib.dump(study,thismodelpath.joinpath(f'study_{study.study_name}.pkl').as_posix())

[I 2024-08-28 18:14:20,185] A new study created in RDB with name: description_countlda_studydefault


No sqlite db found


  0%|          | 0/100 [00:00<?, ?it/s]

c:\Users\lekle\anaconda3\Lib\site-packages\optuna\distributions.py:524: UserWarning: Choices for a categorical distribution should be a tuple of None, bool, int, float and str for persistent storage but contains ['within', 'by', 'also', 'sometimes', 'before', 'thereafter', 'became', 'already', 'thus', 'whereby', 'but', 'herself', 'these', 'formerly', 'itself', 'though', 'her', 'fill', 'him', 'there', 'noone', 'own', 'meanwhile', 'twenty', 'hereupon', 'anyone', 'in', 'every', 'only', 'forty', 'seemed', 'empty', 'something', 'must', 'why', 'thereupon', 'much', 'kg', 'perhaps', 'everywhere', 'wherein', 'several', 'co', 'de', 'more', 'us', 'were', 'an', 'now', 'namely', 'various', 'unless', 'indeed', 'eleven', 'bill', 'himself', 'per', 'using', 'moreover', 'third', 'sometime', 'into', 'would', 'even', 'cannot', 'can', 'serious', 'seeming', 'during', 'twelve', 're', 'don', 'three', 'really', 'at', 'further', 'nothing', 'without', 'yet', 'i', 'so', 'done', 'former', 'if', 'move', 'after', 'b

[I 2024-08-28 18:14:30,554] Trial 14 finished with value: -2269.01182832051 and parameters: {'stop_words': ['within', 'by', 'also', 'sometimes', 'before', 'thereafter', 'became', 'already', 'thus', 'whereby', 'but', 'herself', 'these', 'formerly', 'itself', 'though', 'her', 'fill', 'him', 'there', 'noone', 'own', 'meanwhile', 'twenty', 'hereupon', 'anyone', 'in', 'every', 'only', 'forty', 'seemed', 'empty', 'something', 'must', 'why', 'thereupon', 'much', 'kg', 'perhaps', 'everywhere', 'wherein', 'several', 'co', 'de', 'more', 'us', 'were', 'an', 'now', 'namely', 'various', 'unless', 'indeed', 'eleven', 'bill', 'himself', 'per', 'using', 'moreover', 'third', 'sometime', 'into', 'would', 'even', 'cannot', 'can', 'serious', 'seeming', 'during', 'twelve', 're', 'don', 'three', 'really', 'at', 'further', 'nothing', 'without', 'yet', 'i', 'so', 'done', 'former', 'if', 'move', 'after', 'beyond', 'we', 'becomes', 'not', 'be', 'eg', 'or', 'thick', 'show', 'throughout', 'with', 'many', 'etc', '

['c:/Users/lekle/Projects_Code/HA_MaschinellesLernen_MADS23oB/models/description_countlda_studydefault/study_description_tfidflda_studydefault.pkl']

In [29]:
model = inst_pipeTFLDA(study.best_trial)
fitpipe=model.fit(fulldatadf.description)
joblib.dump(fitpipe,thismodelpath.joinpath(f'fittedpipelinedefault2.pkl').as_posix())

['c:/Users/lekle/Projects_Code/HA_MaschinellesLernen_MADS23oB/models/description_countlda_studydefault/fittedpipelinedefault2.pkl']

In [33]:
#save fitted model
trainpred=fitpipe.transform(xtrain)
testpred=fitpipe.transform(xtest)
predcolumn=fitpipe.transform(fulldatadf.description)
predcolumn=[str(col) for col in predcolumn]

fulldatadf['descr_lda']=predcolumn
fulldatadf.to_csv(thismodelpath.joinpath('transformeddatadefault2.csv',sep=';'))

In [35]:
testpred

array([[0.33333333, 0.33333333, 0.33333333],
       [0.33333333, 0.33333333, 0.33333333],
       [0.33333333, 0.33333333, 0.33333333],
       [0.33333333, 0.33333333, 0.33333333],
       [0.33333333, 0.33333333, 0.33333333],
       [0.33333333, 0.33333333, 0.33333333],
       [0.33333333, 0.33333333, 0.33333333],
       [0.33333333, 0.33333333, 0.33333333],
       [0.33333333, 0.33333333, 0.33333333],
       [0.33333333, 0.33333333, 0.33333333]])

In [31]:
n_top_words = 15 
topics=fitpipe.named_steps['lda'].components_
texts=[[word for word in doc.split()] for doc in xtrain]
# texts=X
dictionary=corpora.Dictionary(texts)
corpus=[dictionary.doc2bow(text) for text in texts]
feature_names = [dictionary[i] for i in range(len(dictionary))]
top_words = []
for topic in topics:
    top_words.append([feature_names[i] for i in topic.argsort()[:-n_top_words - 1:-1]])

In [1]:
# top_words

from sklearn.model_selection import GridSearchCV